In [1]:
#dataset preprocessing

import pandas as pd
import numpy as np
from scipy.stats import norm

df = pd.read_csv('Indian Liver Patient Dataset (ILPD).csv')
 
#add header
headerList = ['age', 'gender', 'total_Bilirubin', 'direct_Bilirubin', 'tp', 'albumin', 'AG_atio', 'SGPT', 'SGOT', 'Alkphos','Target'] 

df.to_csv("trainset.csv", header=headerList, index=False)

In [3]:
# Stats of current dataset
import pandas as pd
import numpy as np
from scipy.stats import norm

df = pd.read_csv('trainset.csv')

# Compute statistics for numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns
numeric_stats = df[numeric_columns].describe()

# Compute statistics for gender column
gender_ratio = df['gender'].value_counts(normalize=True)

# Print the statistics
print("Numeric Columns Statistics:")
print(numeric_stats)

print("\nGender Ratio:")
print(gender_ratio)

Numeric Columns Statistics:
              age  total_Bilirubin  direct_Bilirubin           tp  \
count  582.000000       582.000000        582.000000   582.000000   
mean    44.711340         3.303265          1.488488   290.754296   
std     16.181921         6.213926          2.810324   243.108929   
min      4.000000         0.400000          0.100000    63.000000   
25%     33.000000         0.800000          0.200000   175.250000   
50%     45.000000         1.000000          0.300000   208.000000   
75%     57.750000         2.600000          1.300000   298.000000   
max     90.000000        75.000000         19.700000  2110.000000   

           albumin      AG_atio        SGPT        SGOT     Alkphos  \
count   582.000000   582.000000  582.000000  582.000000  578.000000   
mean     80.824742   110.068729    6.482646    3.141581    0.947145   
std     182.757696   289.141876    1.086306    0.796176    0.319863   
min      10.000000    10.000000    2.700000    0.900000    0.30000

In [ ]:
# do 80 - 20 split but make sure the gender proportion remain the same in both sets